In [ ]:
#%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import math


nruns = 10000

thicknesses = np.arange(5,18,1) #mm
sizes = np.arange(30,61,5) #cm

satls = np.arange(2*1e-3,20*1e-3,5*1e-3) #m
latls = np.arange(2,5,0.5) #m

size = 60
thickness = 10
satl = 10*1e-3  # Short attenuation length
latl = 4  # Long attenuation length

rx, ry, rz = 0.0762/2, 40e-3, 0.0762/2  # PMT radii
area_half_pmt = (math.pi * rx**2)/2 #circle
vol_half_pmt = (4 * math.pi * (((rx*ry)**1.6 + (rx*rz)**1.6 + (ry*rz)**1.6)/3)**(1/1.6))/2

det_times = []
all_times = []
dist = []



for s in sizes:
# for la in latls:

    hit_eff = []
    det_av_time = []
    area_half_wls = ((s*1e-2)**2)/2
    area_total = area_half_wls - area_half_pmt
    #print(area_total)
    rate = (((0.6**2)/2) / (area_half_wls))
    
    for th in thicknesses:
    # for sa in satls:
    
        vol_half_wls = ((th*1e-3) * (s*1e-2)**2)/2
        vol_total = vol_half_wls - vol_half_pmt
        #print(vol_total)

            
        #path = 'outputs/'
        #out_file = path + 'sim_output_' + str(int(thickness*1e+3)) + 'mm_' + str(int(size*1e+2)) + 'cm' + '.txt'
        path = 'outputs/thicknesses/seed(0)/' + str(th) + 'mm_' + str(s) + 'cm_'
        out_file = path + 'sim_output_' + str(satl) + '_m' + str(latl) + '_m.txt'
        #out_file = path + 'sim_output_' + str(satl) + '_m' + str(latl) + '_m.txt'
        out_data = np.genfromtxt(out_file, names=True, delimiter=';', dtype=None, encoding=None) #read data
        hits = np.where(out_data['Hit_coordinate_m'] != '[]')[0]
        det_times_ = out_data[hits]['Time_s']
        all_times_ = out_data['Time_s']
        det_times.append(det_times_)
        all_times.append(all_times_)
        det_av_time.append(np.mean(det_times_))
        hit_eff.append((len(hits))/nruns *100 / rate)
        dist.append(out_data['Total_distance_m'])   

    plt.plot(thicknesses,hit_eff, label = 'Size: ' + str(int(s)) + 'cm')
    # plt.plot(satls,hit_eff, label = 'Long Attenuation Length: ' + str(la) + 'm')

print("number of hits: " + str(len(hits)))
plt.xlabel("Thicknesses (mm)")
# plt.xlabel("Short Attenuation Lengths (m)")
plt.ylabel("Hit Efficiency for Unifrom Photon Flux (%)")
# plt.ylabel("Hit Efficiency (%)")
# plt.title("WLS Plate Attenuation Lengths VS Hit Efficiency\nFor a 60 cm X 60cm X 10mm WLS Plate")
plt.title(f"WLS Plate Thicknesses and Sizes VS Hit Efficiency\n or satl of {str(satl*1e3)}mm and latl of {str(latl)}m")
plt.legend(loc='center right', bbox_to_anchor=(1.57, 0.5))
plt.grid()
plt.show()


In [ ]:
size = 60
thickness = 10
satl = 10*1e-3  # Short attenuation length
latl = 4  # Long attenuation length

rx, ry, rz = 0.0762/2, 40e-3, 0.0762/2  # PMT radii
area_half_pmt = (math.pi * rx**2)/2 #circle
latls2 = [2,2.5,3,3.5,4]
num_total = []

# for la in latls2:

hist = []
det_times = []
num = []

la = 4
    
for s in sizes:

    area_half_wls = ((s*1e-2)**2)/2
    area_total = area_half_wls - area_half_pmt
    rate = (((0.6**2)/2) / (area_half_wls))

    path = 'outputs/thicknesses/seed(0)/' + str(thickness) + 'mm_' + str(s) + 'cm_'
    out_file = path + 'sim_output_' + str(satl) + '_m' + str(la) + '_m.txt'
    out_data = np.genfromtxt(out_file, names=True, delimiter=';', dtype=None, encoding=None) #read data
    hits = np.where(out_data['Hit_coordinate_m'] != '[]')[0]
    det_times.append(out_data[hits]['Time_s'])
    all_times = out_data['Time_s']
    data = out_data[hits]['Time_s']
    bin_size = np.arange(0,15e-9, 1e-9)
    y, binEdges = np.histogram(data, bins=bin_size)
    bincenters = 0.5 * (binEdges[1:] + binEdges[:-1])
    y = (y /rate) 
    i = len(np.where(bincenters <= 0.5e-8)[0])
    a = sum(y[:i])/nruns * 100
    num.append(a)
    hist.append([y, bincenters])
    plt.plot(bincenters, y, label = "Size: " + str(s) + "mm")

    num_total.append(num)
    plt.axvline(x=0.5e-8, linestyle = '--', color = 'grey')
    plt.legend(loc = 'upper right')
    plt.xlabel("Times (s)")
    plt.ylabel("Total hit density for Uniform Photon Flux")
    plt.xlim(0,1.5e-8)
    plt.title(f"Timing Distribution comparing thicknesses for 10000 runs \nfor small atl {str(satl*1e3)}mm, latl {str(la)}m and thickness {str(thickness)}mm")
    plt.grid()
    plt.show()


plt.plot(sizes, num_total[0], label = "Latl: " + str(latls2[0]) + "m")
plt.plot(sizes, num_total[1], label = "Latl: " + str(latls2[1]) + "m")
plt.plot(sizes, num_total[2], label = "Latl: " + str(latls2[2]) + "m")
plt.plot(sizes, num_total[3], label = "Latl: " + str(latls2[3]) + "m")
plt.plot(sizes, num_total[4], label = "Latl: " + str(latls2[4]) + "m")
plt.legend(loc = 'lower right')
plt.grid()
plt.xlabel("Plate thickness (mm)")
plt.ylabel("Hit Efficiency (%)")
plt.title(f"Fraction of hits detected within 5 ns\n for satl {str(satl*1e3)} mm and size {str(size)} cm")
plt.show()


In [ ]:

size = 60
thickness = 10
satl = 7*1e-3  # Short attenuation length
latl = 4.0
rx, ry, rz = 0.0762/2, 40e-3, 0.0762/2  # PMT radii
area_half_pmt = (math.pi * rx**2)/2 #circle
hist = []
det_times = []
latls = np.arange(2,5,0.5) #m
num = []

# for th in thicknesses:
for sa in satls:

    area_half_wls = ((size*1e-2)**2)/2
    area_total = area_half_wls - area_half_pmt

    path = 'outputs/atls/seed(1)/' + str(thickness) + 'mm_' + str(size) + 'cm_'
    out_file = path + 'sim_output_' + str(sa) + '_m' + str(latl) + '_m.txt'
    out_data = np.genfromtxt(out_file, names=True, delimiter=';', dtype=None, encoding=None) #read data
    hits = np.where(out_data['Hit_coordinate_m'] != '[]')[0]
    det_times.append(out_data[hits]['Time_s'])
    all_times = out_data['Time_s']
    bin_size = np.arange(0,20e-9, 1e-9)
    y, binEdges = np.histogram(out_data[hits]['Time_s'], bins=bin_size)
    bincenters = 0.5 * (binEdges[1:] + binEdges[:-1])
    y = (y)
    i = len(np.where(bincenters <= 0.5e-8)[0])
    a = sum(y[:i])/nruns * 100
    num.append(a)
    hist.append([y, bincenters])
    plt.plot(bincenters, y, label = "Satl: " + str(sa) + "mm")
    
plt.axvline(x=0.5e-8, linestyle = '--', color = 'grey')
plt.legend(loc = 'upper right')
plt.xlabel("Times (s)")
plt.ylabel("Total hit density for Uniform Photon Flux")
plt.title("Timing Distribution comparing satls \nfor long atl 4m, size 60cm and thickness 10mm")
plt.grid()
plt.show()

plt.plot(satls,num)
plt.legend(loc = 'lower right')
plt.grid()
plt.xlabel("Plate sizes (cm)")
plt.ylabel("Hit Efficiency (%)")
plt.title(f"Fraction of hits detected within 5 ns\n for satl {str(satl*1e3)} mm latl {str(latl)} and size {str(size)} mm")
plt.show()




In [ ]:
plt.hist(det_times[10 + (len(thicknesses)*1)], bins=100, label = "Size: " + str(int(sizes[1]*1e+2)) + "cm", histtype="step")
plt.hist(det_times[10 + (len(thicknesses)*9)], bins=100, label = "Size: " + str(int(sizes[9]*1e+2)) + "cm", histtype="step")
       
plt.title("Detected Timing Distribution - Thickness: " + str(thicknesses[10]) + "m")
plt.grid()
plt.ylabel("Frequency")
plt.xlabel("Time (s)")
plt.legend(loc = "upper right")
plt.show()


for s in range(0,len(sizes)):
    plt.hist(det_times[6 + (len(thicknesses)*s)], bins=100, label = "Size: " + str(int(sizes[s]*1e+2)) + "cm")

plt.title("Detected Timing Distribution - Thickness: " + str(thicknesses[6]) + "m")
plt.legend(loc="upper right")
plt.grid()
plt.show()

In [ ]:
plt.hist(all_times[6 + (len(thicknesses)*1)], bins=100, label = "Size: " + str(int(sizes[1]*1e+2)) + "cm", histtype="step")
plt.hist(all_times[6 + (len(thicknesses)*9)], bins=100, label = "Size: " + str(int(sizes[9]*1e+2)) + "cm", histtype="step")
       
plt.title("All Timing Distribution - Thickness: " + str(thicknesses[6]) + "m")
plt.grid()
plt.ylabel("Frequency")
plt.xlabel("Time (s)")
plt.legend(loc = "upper right")
plt.show()


for s in range(0,len(sizes)):
    plt.hist(all_times[6 + (len(thicknesses)*s)], bins=100, label = "Size: " + str(int(sizes[s]*1e+2)) + "cm")

plt.title("All Timing Distribution - Thickness: " + str(thicknesses[6]) + "m")
plt.legend(loc="upper right")
plt.grid()
plt.show()

In [ ]:
plt.hist(dist[6 + (len(thicknesses)*1)], bins=100, label = "Size: " + str(int(sizes[1]*1e+2)) + "cm", histtype="step")
plt.hist(dist[6 + (len(thicknesses)*9)], bins=100, label = "Size: " + str(int(sizes[9]*1e+2)) + "cm", histtype="step")
       
plt.title("All Distance Distribution - Thickness: " + str(thicknesses[6]) + "m")
plt.grid()
plt.ylabel("Frequency")
plt.xlabel("Distance (m)")
plt.legend(loc = "upper right")
plt.show()


for s in range(0,len(sizes)):
    plt.hist(dist[6 + (len(thicknesses)*s)], bins=100, label = "Size: " + str(int(sizes[s]*1e+2)) + "cm")

plt.title("All Distance Distribution - Thickness: " + str(thicknesses[6]) + "m")
plt.legend(loc="upper right")
plt.grid()
plt.show()